In [ ]:
import boto3

kda_client = boto3.client('kinesisanalytics')
kinesis = boto3.client('kinesis')
iam = boto3.client('iam')

## IAM Role and Policy

In [ ]:
assume_role_policy_document="""{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "kinesisanalytics.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
"""

In [ ]:
cr = iam.create_role(
    RoleName='kda-sample-role',
    Path='/service-role/',
    AssumeRolePolicyDocument=assume_role_policy_document
)

print(cr)

In [ ]:
import os
account_no = os.environ['PA_ACCOUNT_NO']

In [ ]:
policy_document="""{
            "Version": "2012-10-17",
            "Statement": [
               
                {
                    "Sid": "ListCloudwatchLogGroups",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogGroups"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-west-2:""" + account_no + """:log-group:*"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogStreams",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogStreams"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-west-2:""" + account_no + """:log-group:/aws/kinesis-analytics/sample:log-stream:*"
                    ]
                },
                {
                    "Sid": "PutCloudwatchLogs",
                    "Effect": "Allow",
                    "Action": [
                        "logs:PutLogEvents"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-west-2:""" + account_no + """:log-group:/aws/kinesis-analytics/sample:log-stream:kinesis-analytics-log-stream"
                    ]
                },
                {
                
                    "Sid": "ReadInputStream",
                    "Effect": "Allow",
                    "Action": [
                        "kinesis:*"
                    ],
                    "Resource": [
                        "arn:aws:kinesis:us-west-2:""" + account_no + """:stream/pyth"
                    ]
                },
                {
                    "Sid": "UseLambdaFunction",
                    "Effect": "Allow",
                    "Action": [
                        "lambda:InvokeFunction",
                        "lambda:GetFunctionConfiguration"
                    ],
                    "Resource": [
                        "arn:aws:lambda:us-west-2:""" + account_no + """:function:postproc-dev-pp:$LATEST"
                    ]
                }
            ]
}"""

In [ ]:
cp = iam.create_policy(
    PolicyName='kda-sample-app',
    Path='/service-role/',
    PolicyDocument=policy_document
)

print(cp)

In [ ]:
ap = iam.attach_role_policy(
    RoleName='kda-sample-role',
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-sample-app".format(account_no)
)

## Application Definition

In [ ]:
ca = kda_client.create_application(
    ApplicationName='pex'
)
print(ca)

In [ ]:
app_desc = kda_client.describe_application(
    ApplicationName='pex'
)
print(app_desc)

In [ ]:
kda_client.add_application_input(
    ApplicationName='pex',
    CurrentApplicationVersionId=app_desc['ApplicationDetail']['ApplicationVersionId'],
    Input={
        'NamePrefix':'SOURCE_SQL_STREAM',
        'KinesisStreamsInput': {
            'ResourceARN': "arn:aws:kinesis:us-west-2:" + account_no + ":stream/pyth",
            'RoleARN': 'arn:aws:iam::{}:role/service-role/kda-sample-role'.format(account_no),
        },
        'InputSchema': {
            "RecordFormat":{
                "RecordFormatType":"JSON",
                "MappingParameters":{
                   "JSONMappingParameters":{
                      "RecordRowPath":"$"
                   }
                }
             },
             "RecordEncoding":"UTF-8",
             "RecordColumns":[
                {
                   "Name":"symbol",
                   "Mapping":"$.symbol",
                   "SqlType":"VARCHAR(32)"
                },
                {
                   "Name":"price",
                   "Mapping":"$.price",
                   "SqlType":"DOUBLE"
                },
                {
                   "Name":"confidence",
                   "Mapping":"$.confidence",
                   "SqlType":"DOUBLE"
                }
             ]
        },
        "InputParallelism":{
            "Count":1
        }
    }
)

In [ ]:
app_code="""CREATE OR REPLACE STREAM "DESTINATION_SQL_STREAM" ("symbol" VARCHAR(32), "price" DOUBLE, "confidence" DOUBLE);
CREATE OR REPLACE  PUMP "STREAM_PUMP" AS INSERT INTO "DESTINATION_SQL_STREAM"
SELECT STREAM "symbol",
LAST_VALUE("price") OVER W1 AS "price",
LAST_VALUE("confidence") OVER W1 AS "confidence"
FROM "SOURCE_SQL_STREAM_001"
WINDOW W1 AS (
    PARTITION BY "symbol"
    RANGE INTERVAL '10' SECOND PRECEDING
);
"""

In [ ]:
app_desc = kda_client.describe_application(
    ApplicationName='pex'
)

ua = kda_client.update_application(
    ApplicationName='pex',
    CurrentApplicationVersionId=app_desc['ApplicationDetail']['ApplicationVersionId'],
    ApplicationUpdate={
        'ApplicationCodeUpdate':app_code
    }
)

print(ua)

In [ ]:
app_desc = kda_client.describe_application(
    ApplicationName='pex'
)

print(app_desc)

In [ ]:
app_desc = kda_client.describe_application(
    ApplicationName='pex'
)

ao = kda_client.add_application_output(
    ApplicationName='pex',
    CurrentApplicationVersionId=app_desc['ApplicationDetail']['ApplicationVersionId'],
    Output={
        'Name':'DESTINATION_SQL_STREAM',
        'LambdaOutput': {
            "ResourceARN":"arn:aws:lambda:us-west-2:" + account_no + ":function:postproc-dev-pp:$LATEST",
            'RoleARN': 'arn:aws:iam::{}:role/service-role/kda-sample-role'.format(account_no)
        },
        'DestinationSchema': {
            'RecordFormatType': 'JSON'
        }
    }
)

print(ao)

In [ ]:
input_id = app_desc['ApplicationDetail']['InputDescriptions'][0]['InputId']
print(input_id)

In [ ]:
kda_client.start_application(
    ApplicationName='pex',
    InputConfigurations=[
        {
            'Id': input_id,
            'InputStartingPositionConfiguration': {
                'InputStartingPosition':'NOW'
            }
        }
    ]
)

In [ ]:
kda_client.describe_application(
    ApplicationName='pex'
)['ApplicationDetail']['ApplicationStatus']

## Clean Up

### Application

In [ ]:
app_desc = kda_client.describe_application(
    ApplicationName='pex'
)
print(app_desc)

In [ ]:
kda_client.delete_application(
    ApplicationName='pex',
    CreateTimestamp=app_desc['ApplicationDetail']['CreateTimestamp']
)

### IAM

In [ ]:
policy_arn="arn:aws:iam::{}:policy/service-role/kda-sample-app".format(account_no)
print(policy_arn)

In [ ]:
iam.detach_role_policy(
    RoleName='kda-sample-role',
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_policy(
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_role(
    RoleName='kda-sample-role'
)